## DEBUG

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import logging
import matplotlib.pyplot as plt

import numpy as np

from pkg import c, m, f 
import torch
import logging
from pkg import c, m, f 


In [7]:
# Config
cfg = {
    'batch_size': 100,
    'threshold_value': 10,
    'num_epochs': 10,
    'learning_rate': 0.001,
    'classes': 2
}

# classes 
paths = c.PathManager()
data = c.PeakImageDataset(paths)
prep = c.DataPreparation(paths, data, batch_size=cfg['batch_size'])
water_h5 = data.load_h5(paths.water_background_h5)
ip = c.ImageProcessor(water_h5)
p = c.PeakThresholdProcessor(threshold_value=cfg['threshold_value'])

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())



Number of peak images: 947
Number of water images: 947
Number of label images: 947
<class 'numpy.ndarray'>
True


In [3]:
# if not already generated 
# ip.process_directory(paths, p.threshold_value) 
print(dir(m))

['BasicCNN1', 'BasicCNN2', 'DenseNet121_Weights', 'DenseNetBraggPeakClassifier', 'F', 'ResNet50BraggPeakClassifier', 'ResNet50_Weights', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'models', 'nn', 'np', 'optim', 'os', 'torch']


In [8]:
models = [
    m.BasicCNN1(),
    m.BasicCNN2(),
    m.ResNet50BraggPeakClassifier(),
    m.DenseNetBraggPeakClassifier()
]

for i in models:
    model = i.to(device)
    # loss function/ combines a Sigmoid layer and the BCELoss in one single class
    criterion = nn.BCEWithLogitsLoss()
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg['learning_rate'])
    print("Criterion: ", criterion)
    print("Optimizer: ", optimizer)
    print("Learning rate: ", optimizer.param_groups[0]['lr'])
    # data loaders 
    train_loader, test_loader = prep.prep_data()
    loader = [train_loader, test_loader]
    N = [len(loader[0].dataset), len(loader[1].dataset)]

    epochs = cfg['num_epochs']
    batch = [len(loader[0]), len(loader[1])]
    classes = cfg['classes']

    f.train_test_model(model, loader, criterion, optimizer, epochs, device, N, batch, classes)
    
    model.cpu()  # Move model back to CPU to free up GPU memory
    torch.cuda.empty_cache()  # Clear unused memory

Criterion:  BCEWithLogitsLoss()
Optimizer:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)
Learning rate:  0.001
Data prepared.
Train size: 757
Test size: 190
Batch size: 100
Number of batches in train_loader: 8 


Model: BasicCNN1
Training and testing the model...
-- epoch 0


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.68 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 58.69 GiB is allocated by PyTorch, and 25.28 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)